<a href="https://colab.research.google.com/github/sarthag/CIFAR10/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import time

In [ ]:
from google.colab import drive
import sys
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/CIFAR10') #need to fix the module importing 

Mounted at /content/gdrive


In [ ]:
transform = transforms.Compose(
                [transforms.ToTensor(),
                 transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                ])

In [ ]:
in_channel = 3
num_classes = 10
learning_rate = 0.0005
batch_size = 16
num_epochs = 100

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def basic_test(Architecture):
    start = time.process_time()
    model = Architecture()
    x = torch.randn(16,3,32,32)
    print(model(x).shape)
    end = time.process_time()
    print("time: ", end-start)

In [ ]:
class Architecture1(nn.Module):
    def __init__(self, num_classes = num_classes, in_channel = 3):
        super(Architecture1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 5, stride = 1, padding = 2)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size= (2,2), stride = (2,2))
        self.fc1 = nn.Linear(32*4*4, 120)
        self.output = nn.Linear(120, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #16
        x = self.pool(F.relu(self.conv2(x))) #8
        x = self.pool(F.relu(self.conv3(x))) #4
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.output(x)
        
        return x

In [ ]:
basic_test(Architecture1)

torch.Size([16, 10])
time:  0.018689850999999535


In [ ]:
class Architecture2(nn.Module):
    def __init__(self, num_classes = num_classes, in_channel = 3):
        super(Architecture2, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.pool = nn.MaxPool2d(kernel_size= (2,2), stride = (2,2))
        self.fc1 = nn.Linear(32*4*4, 120)
        self.output = nn.Linear(120, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #16
        x = self.pool(F.relu(self.conv2(x))) #8
        x = self.pool(F.relu(self.conv3(x))) #4
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = self.output(x)
        
        return x

In [ ]:
basic_test(Architecture2)

torch.Size([16, 10])
time:  0.009469259000001173


In [ ]:
class Architecture3(nn.Module):
    def __init__(self, num_classes = num_classes, in_channel = 3):
        super(Architecture3, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = 5, stride = 1, padding = 2)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size= (2,2), stride = (2,2))
        self.fc1 = nn.Linear(32*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.output = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #16
        x = self.pool(F.relu(self.conv2(x))) #8
        x = self.pool(F.relu(self.conv3(x))) #4
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.output(x)
        
        return x

In [ ]:
basic_test(Architecture3)

torch.Size([16, 10])
time:  0.009873068000000984


In [ ]:
loss_fun = nn.CrossEntropyLoss()

In [ ]:
def check_accuracy(loader, model):
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            scores = model(x)
            predictions = scores.argmax(1)
            num_correct += sum((predictions == y))
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)

In [ ]:
def model_test(model):
    model = model
    optimizer = optim.Adam(model.parameters(), lr = learning_rate) 
    for epoch in range(num_epochs):
        start = time.process_time()
        loss_train = 0
        for batch, (data,targets) in enumerate(train_loader):

            #fw_prop
            scores = model(data)
            loss = loss_fun(scores, targets)

            #bw_prop
            optimizer.zero_grad()
            loss.backward()
            loss_train += loss.item()

            #Optimizer
            optimizer.step()
        avg_epoch_loss = loss_train/(len(train_loader))
        end = time.process_time()
        exec_time = end-start
        print('Epoch ({}/{}),Training loss : {}, Time:{}'.format(epoch+1,num_epochs,avg_epoch_loss,exec_time))
        
    print("accuracy: ", check_accuracy(train_loader, model))
    return model

In [ ]:
t1 = time.process_time()
prelim1 = model_test(Architecture1())
t2 = time.process_time()
print("Total time: ", t2-t1)

Epoch (1/100),Training loss : 1.5400535319709778, Time:36.838177566
Epoch (2/100),Training loss : 1.176681277475357, Time:37.473556435999996
Epoch (3/100),Training loss : 1.0186497174263, Time:37.419714295000006
Epoch (4/100),Training loss : 0.9198260778903962, Time:37.00605273699999
Epoch (5/100),Training loss : 0.84677586540699, Time:37.25461439699998
Epoch (6/100),Training loss : 0.7847634663534164, Time:37.079242970999985
Epoch (7/100),Training loss : 0.7356908819222451, Time:37.19799769900001
Epoch (8/100),Training loss : 0.6897472673177719, Time:37.136718273999975
Epoch (9/100),Training loss : 0.6518433056521415, Time:36.95782384399996
Epoch (10/100),Training loss : 0.6092366298699379, Time:36.846032889000014
Epoch (11/100),Training loss : 0.576672398828268, Time:36.635999431000016
Epoch (12/100),Training loss : 0.5432141628101468, Time:36.47560265499999
Epoch (13/100),Training loss : 0.5114823631894588, Time:36.589850259
Epoch (14/100),Training loss : 0.4845656049931049, Time:36

In [ ]:
t1 = time.process_time()
prelim2 = model_test(Architecture2())
t2 = time.process_time()
print("Total time: ", t2-t1)

Epoch (1/100),Training loss : 1.5529216631698608, Time:32.53574016399989
Epoch (2/100),Training loss : 1.231470044517517, Time:32.47905598999978
Epoch (3/100),Training loss : 1.0917365889167785, Time:32.58090950699989
Epoch (4/100),Training loss : 0.9906432021617889, Time:32.51626763000013
Epoch (5/100),Training loss : 0.9146503470134735, Time:32.51414180200027
Epoch (6/100),Training loss : 0.8527795765161514, Time:32.25898286699976
Epoch (7/100),Training loss : 0.7967887655067444, Time:32.30826390200036
Epoch (8/100),Training loss : 0.7501671705961227, Time:32.25525009800003
Epoch (9/100),Training loss : 0.7117111773777008, Time:32.02299744099946
Epoch (10/100),Training loss : 0.6732510753440857, Time:31.95604847300001
Epoch (11/100),Training loss : 0.6376587969779969, Time:31.963600927000698
Epoch (12/100),Training loss : 0.6062960350942612, Time:32.13303679300043
Epoch (13/100),Training loss : 0.5762286661672592, Time:31.89300535199982
Epoch (14/100),Training loss : 0.54735987906456

In [ ]:
t1 = time.process_time()
prelim3 = model_test(Architecture3())
t2 = time.process_time()
print("Total time: ", t2-t1)

Epoch (1/100),Training loss : 1.5944657737731933, Time:35.61249239499921
Epoch (2/100),Training loss : 1.2723376806259155, Time:35.925563764999424
Epoch (3/100),Training loss : 1.1140238429927827, Time:35.98345769000025
Epoch (4/100),Training loss : 1.0020751293182373, Time:35.960485246000644
Epoch (5/100),Training loss : 0.9224431224679946, Time:35.753280524999354
Epoch (6/100),Training loss : 0.8586114419984817, Time:36.18854103400008
Epoch (7/100),Training loss : 0.8069331151580811, Time:35.840139789000204
Epoch (8/100),Training loss : 0.7654355543994904, Time:35.72124048000023
Epoch (9/100),Training loss : 0.7235511983752251, Time:35.85822074299995
Epoch (10/100),Training loss : 0.6872760372066498, Time:35.824409621000086
Epoch (11/100),Training loss : 0.6533809050893784, Time:35.839144504000615
Epoch (12/100),Training loss : 0.6184472331213952, Time:35.77696208399993
Epoch (13/100),Training loss : 0.5881413195943832, Time:35.946864181999445
Epoch (14/100),Training loss : 0.5613671

In [ ]:
#for test
model = prelim3
print("accuracy: ", check_accuracy(test_loader, model))

accuracy:  0.6588


In [ ]:
model = prelim2
print("accuracy: ", check_accuracy(test_loader, model))

accuracy:  0.6659


In [ ]:
model = prelim1
print("accuracy: ", check_accuracy(test_loader, model))

accuracy:  0.6659
